# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-19 04:13:45] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=36054, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=993848376, constrai

[2025-07-19 04:13:57] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-07-19 04:13:57] Init torch distributed begin.


[2025-07-19 04:13:58] Init torch distributed ends. mem usage=0.00 GB


[2025-07-19 04:13:59] Load weight begin. avail mem=77.99 GB


[2025-07-19 04:13:59] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-19 04:14:03] INFO:     Started server process [284450]
[2025-07-19 04:14:03] INFO:     Waiting for application startup.
[2025-07-19 04:14:03] INFO:     Application startup complete.
[2025-07-19 04:14:03] INFO:     Uvicorn running on http://0.0.0.0:36054 (Press CTRL+C to quit)


[2025-07-19 04:14:04] INFO:     127.0.0.1:34996 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-19 04:14:04] INFO:     127.0.0.1:35010 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-19 04:14:04] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:04.866619


[2025-07-19 04:14:06] INFO:     127.0.0.1:35016 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:06] The server is fired up and ready to roll!


Server started on http://localhost:36054


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-19 04:14:09] INFO:     127.0.0.1:35030 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-07-19 04:14:09] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:09.476993


[2025-07-19 04:14:09] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.18, #queue-req: 0, timestamp: 2025-07-19T04:14:09.832559
[2025-07-19 04:14:09] INFO:     127.0.0.1:35038 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-07-19 04:14:09] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:09.870308


[2025-07-19 04:14:10] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, cuda graph: False, gen throughput (token/s): 90.53, #queue-req: 0, timestamp: 2025-07-19T04:14:10.274413
[2025-07-19 04:14:10] INFO:     127.0.0.1:35042 - "POST /generate HTTP/1.1" 200 OK


[2025-07-19 04:14:10] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 75, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:10.314995


[2025-07-19 04:14:10] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: False, gen throughput (token/s): 67.36, #queue-req: 0, timestamp: 2025-07-19T04:14:10.868238
[2025-07-19 04:14:10] INFO:     127.0.0.1:35046 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-07-19 04:14:10] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:10.942970
[2025-07-19 04:14:10] INFO:     127.0.0.1:35052 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:10] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:10.983009


[2025-07-19 04:14:11] INFO:     127.0.0.1:35054 - "POST /generate HTTP/1.1" 200 OK


[2025-07-19 04:14:11] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:11.144887


[2025-07-19 04:14:11] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 51.36, #queue-req: 0, timestamp: 2025-07-19T04:14:11.666479
[2025-07-19 04:14:11] INFO:     127.0.0.1:35070 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-07-19 04:14:11] Prefill batch. #new-seq: 2, #new-token: 70, #cached-token: 28, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:11.814389


[2025-07-19 04:14:12] Decode batch. #running-req: 2, #token: 116, token usage: 0.01, cuda graph: False, gen throughput (token/s): 108.97, #queue-req: 0, timestamp: 2025-07-19T04:14:12.308857


[2025-07-19 04:14:12] Decode batch. #running-req: 2, #token: 196, token usage: 0.01, cuda graph: False, gen throughput (token/s): 132.51, #queue-req: 0, timestamp: 2025-07-19T04:14:12.912571


[2025-07-19 04:14:13] Decode batch. #running-req: 2, #token: 276, token usage: 0.01, cuda graph: False, gen throughput (token/s): 133.67, #queue-req: 0, timestamp: 2025-07-19T04:14:13.511039


[2025-07-19 04:14:13] INFO:     127.0.0.1:35090 - "POST /generate HTTP/1.1" 200 OK


[2025-07-19 04:14:14] Decode batch. #running-req: 1, #token: 200, token usage: 0.01, cuda graph: False, gen throughput (token/s): 106.08, #queue-req: 0, timestamp: 2025-07-19T04:14:14.095487
[2025-07-19 04:14:14] INFO:     127.0.0.1:35074 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:14] Prefill batch. #new-seq: 1, #new-token: 317, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:14.217063


[2025-07-19 04:14:14] Decode batch. #running-req: 1, #token: 389, token usage: 0.02, cuda graph: False, gen throughput (token/s): 59.90, #queue-req: 0, timestamp: 2025-07-19T04:14:14.763295


[2025-07-19 04:14:15] Decode batch. #running-req: 1, #token: 429, token usage: 0.02, cuda graph: False, gen throughput (token/s): 66.49, #queue-req: 0, timestamp: 2025-07-19T04:14:15.364869


[2025-07-19 04:14:15] Decode batch. #running-req: 1, #token: 469, token usage: 0.02, cuda graph: False, gen throughput (token/s): 68.51, #queue-req: 0, timestamp: 2025-07-19T04:14:15.948711


[2025-07-19 04:14:16] INFO:     127.0.0.1:46588 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-07-19 04:14:16] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:16.465077


[2025-07-19 04:14:17] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, cuda graph: False, gen throughput (token/s): 24.98, #queue-req: 0, timestamp: 2025-07-19T04:14:17.549966
[2025-07-19 04:14:17] INFO:     127.0.0.1:46592 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-07-19 04:14:17] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:17.807014


[2025-07-19 04:14:18] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, cuda graph: False, gen throughput (token/s): 67.41, #queue-req: 0, timestamp: 2025-07-19T04:14:18.143317


[2025-07-19 04:14:18] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, cuda graph: False, gen throughput (token/s): 79.50, #queue-req: 0, timestamp: 2025-07-19T04:14:18.646481


[2025-07-19 04:14:19] Decode batch. #running-req: 1, #token: 150, token usage: 0.01, cuda graph: False, gen throughput (token/s): 68.25, #queue-req: 0, timestamp: 2025-07-19T04:14:19.232535
[2025-07-19 04:14:19] INFO:     127.0.0.1:46594 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-07-19 04:14:19] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:19.274542
[2025-07-19 04:14:19] INFO:     127.0.0.1:46604 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-07-19 04:14:19] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:19.420808
[2025-07-19 04:14:19] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 34, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-19T04:14:19.423152


 33%|███▎      | 1/3 [00:00<00:00,  5.91it/s]

 67%|██████▋   | 2/3 [00:00<00:00,  9.38it/s]

100%|██████████| 3/3 [00:00<00:00, 15.13it/s]

[2025-07-19 04:14:19] INFO:     127.0.0.1:46620 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:19] INFO:     127.0.0.1:46634 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:19] INFO:     127.0.0.1:46612 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-07-19 04:14:19] INFO:     127.0.0.1:46650 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:19] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:19.628482
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-07-19 04:14:19] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:19.766223


[2025-07-19 04:14:19] INFO:     127.0.0.1:46656 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-19 04:14:31] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=33317, nccl_port=None, mem_fraction_static=0.7835956249999999, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_se

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-19 04:14:33] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-19 04:14:34] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-19 04:14:43] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-19 04:14:43] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-19 04:14:43] Init torch distributed begin.


...

[2025-07-19 04:14:50] INFO:     Started server process [287356]
[2025-07-19 04:14:50] INFO:     Waiting for application startup.
[2025-07-19 04:14:50] INFO:     Application startup complete.
[2025-07-19 04:14:50] INFO:     Uvicorn running on http://0.0.0.0:33317 (Press CTRL+C to quit)


[2025-07-19 04:14:51] INFO:     127.0.0.1:43814 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-19 04:14:51] INFO:     127.0.0.1:43824 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-19 04:14:51] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:51.590107


[2025-07-19 04:14:52] INFO:     127.0.0.1:43834 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 04:14:52] The server is fired up and ready to roll!


Server started on http://localhost:33317


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-19 04:14:56] INFO:     127.0.0.1:58080 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-07-19 04:14:56] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T04:14:56.576585


[2025-07-19 04:14:57] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 5.48, #queue-req: 0, timestamp: 2025-07-19T04:14:57.400556


[2025-07-19 04:14:58] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 59.69, #queue-req: 0, timestamp: 2025-07-19T04:14:58.070669
[2025-07-19 04:14:58] INFO:     127.0.0.1:58086 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-07-19 04:14:58] Child process unexpectedly failed with exitcode=9. pid=287641
[2025-07-19 04:14:58] Child process unexpectedly failed with exitcode=9. pid=287507
